In [1]:
import os
import sys
import time
import yaml
import pandas as pd
import numpy as np
import re

with open('../../config.local.yaml', 'r') as f:
    local_config = yaml.safe_load(f)

LOCAL_PATH = local_config['LOCAL_PATH']

sys.path.append(os.path.join(LOCAL_PATH, "src/python"))

import data_tools as dt
from utils import parse_casenum

from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from IPython.core.display import HTML
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from stargazer.stargazer import Stargazer

rng = np.random.default_rng(12898)

SUPPORT_TAGS = ['DEFINITELY SUPPORT', 'SOMEWHAT SUPPORT']
OPPOSE_TAGS = ['DEFINITELY OPPOSE', 'SOMEWHAT OPPOSE']
ORDERED_TAGS = ['DEFINITELY SUPPORT', 'SOMEWHAT SUPPORT', 'SOMEWHAT OPPOSE', 'DEFINITELY OPPOSE']

APPROVE_TAGS = ['APPROVED', 'APPROVED IN PART OR WITH MODIFICATIONS']



## Data Preparation

In [2]:
dfa = dt.get_agenda_items(verbose=False)
dfm = dt.get_minutes(verbose=False)
dfl = dt.get_supplemental_docs(verbose=False)

In [3]:
# Keep only applications not withdrawn
idx = dfm['project_result']!='APPLICATION WITHDRAWN'
dfm = dfm.loc[idx].reset_index(drop=True)

In [4]:
# Keep only supporting or opposing letters
idx = dfl['support_or_oppose'].isin(SUPPORT_TAGS + OPPOSE_TAGS)
dfl = dfl.loc[idx].reset_index(drop=True)

In [5]:
# Count support and opposition for minutes items
for so in ['support', 'oppose']:
    for typ in ['', 'individual', 'official', 'other']:
        dfm[f'n_{typ}_{so}'] = 0
        
for idx, irow in dfm.iterrows():
    date = irow['date']
    item_no = irow['item_no']
    for jdx, jrow in dfl.loc[dfl['date']==date].iterrows():
        referenced_items = [s.strip() for s in jrow['referenced_items'].split(',')]
        is_so = {}
        is_so['support'] = jrow['support_or_oppose'] in SUPPORT_TAGS
        is_so['oppose'] = jrow['support_or_oppose'] in OPPOSE_TAGS
        is_type = {}
        is_type[''] = 1
        is_type['individual'] = 1*(jrow['author_type']=='INDIVIDUAL')
        is_type['official'] = 1*(jrow['author_type']=='PUBLIC OFFICIAL')
        is_type['other'] = 1 - is_type['individual'] - is_type['official']
        if item_no in referenced_items:
            for so in ['support', 'oppose']:
                for typ in ['', 'individual', 'official', 'other']:
                    dfm.loc[idx, f'n_{typ}_{so}'] += is_type[typ]*is_so[so]

In [6]:
# Deliberation order for minutes items
dfm['deliberation_order'] = dfm.groupby('date')[
    'minutes_start_line'
].rank(method='dense', ascending=True)

In [7]:
# Agenda item order
def split_item_no(x):
    match = re.match(r'(\d+)([a-zA-Z]*)', x)
    if match:
        return int(match.group(1)), match.group(2)
    else:
        return int(match.group(1)), ''

dfm[['item_no_num', 'item_no_sfx']] = dfm['item_no'].apply(lambda x: pd.Series(split_item_no(x)))

dfm['agenda_order'] = (
    dfm.sort_values(by=['date','item_no_num','item_no_sfx'])
       .groupby(['date'])
       .cumcount()+1
)

In [8]:
# Total number of agenda items for each meeting
tempdf = dfa.groupby('date').agg(num_agenda_items=('item_no','count')).reset_index()
dfm = dfm.merge(tempdf, on='date', how='left')

In [9]:
# consent calendar indicator
tempdf = dfa[['date', 'item_no', 'is_cc_part']]
dfm = dfm.merge(tempdf, on=['date', 'item_no'], how='left')
dfm['consent_calendar'] = 1*dfm['is_cc_part']

In [10]:
# agenda perplexity
tempdf = dfa[['date', 'item_no', 'perplexity']].rename(columns={'perplexity': 'agenda_perplexity'})
dfm = dfm.merge(tempdf, on=['date', 'item_no'], how='left')

In [11]:
# time to act
tempdf = dfa[['date', 'item_no', 'last_day_to_act']].copy()
tempdf['time_to_act'] = (tempdf['last_day_to_act'] - pd.to_datetime(tempdf['date'], errors='coerce')) / pd.to_timedelta(1,unit='D')
dfm = dfm.merge(tempdf, on=['date', 'item_no'], how='left')

In [12]:
# appealed
tempdf = dfa[['date', 'item_no', 'appealed']].copy()
dfm = dfm.merge(tempdf, on=['date', 'item_no'], how='left')
dfm['appealed'] = 1*dfm['appealed']

In [13]:
# number laws
tempdf = dfa[['date', 'item_no', 'relevant_laws']].copy()
tempdf['num_referenced_laws'] = tempdf['relevant_laws'].apply(lambda x: len(x.split('\n')))
dfm = dfm.merge(tempdf, on=['date', 'item_no'], how='left')

In [ ]:
# case suffixes
suffixes = set()
for idx, row in dfm.iterrows():
    parsed_casenum = parse_casenum(row['title'])
    for sfx in parsed_casenum['suffixes']:
        suffixes.add(sfx)
sfx_cols = []
for sfx in suffixes:
    mycol = f'sfx_{sfx}'
    sfx_cols.append(mycol)
    dfm[mycol] = 0
for idx, row in dfm.iterrows():
    parsed_casenum = parse_casenum(row['title'])
    for sfx in parsed_casenum['suffixes']:
        dfm.loc[idx, f'sfx_{sfx}'] = 1

In [ ]:
# Export data to CSV
df_out = dfm.copy().rename(columns={
    'perplexity': 'minutes_perplexity',
})
mycols = [
    'year', 'date', 'item_no', 'title', 'related_cases',
    'agenda_content', 'agenda_perplexity', 'agenda_order', 'num_agenda_items', 'consent_calendar',
    'minutes_content', 'minutes_perplexity', 'agenda_item_summary', 'deliberations_summary', 'motion_summary',
    'appealed', 'relevant_laws', 'num_referenced_laws',
    'moved', 'seconded', 'ayes', 'nays', 'abstained', 'recused', 'absent', 'vote_result', 
    'n_ayes', 'n_nays', 'n_abstained', 'n_recused', 'n_absent', 
    'appeal_result', 'project_result', 
    'n__support', 'n__oppose', 'n_individual_support', 'n_individual_oppose', 
    'n_official_support', 'n_official_oppose', 'n_other_support', 'n_other_oppose'
]
mycols = mycols + sfx_cols
df_out[mycols].to_csv(
    os.path.join(LOCAL_PATH, "intermediate_data/cpc/export-for-joe.csv"), 
    header=True, 
    index=False
)

In [16]:
suffixes

{'1A',
 'ACI',
 'AD',
 'AMDT1',
 'AMDT2',
 'AMDT3',
 'BL',
 'BSA',
 'CA',
 'CCMP',
 'CDO',
 'CDP',
 'CN',
 'COA',
 'CPIO',
 'CPIOA',
 'CPU',
 'CU',
 'CU2',
 'CU3',
 'CUB',
 'CUW',
 'CUX',
 'DA',
 'DB',
 'DD',
 'DI',
 'DRB',
 'ELD',
 'F',
 'GB',
 'GPA',
 'GPAJ',
 'HCA',
 'HD',
 'M1',
 'M3',
 'MCUP',
 'MEL',
 'MSC',
 'MSP',
 'PA1',
 'PA2',
 'PAB',
 'PHP',
 'PMEX',
 'PMLA',
 'PR',
 'PSH',
 'PUB',
 'RDP',
 'SCEA',
 'SIP',
 'SL',
 'SN',
 'SP',
 'SPE',
 'SPP',
 'SPPA',
 'SPPC',
 'SPPE',
 'SPR',
 'SUD',
 'TDR',
 'TOC',
 'VCU',
 'VHCA',
 'VZC',
 'VZCJ',
 'WDI',
 'ZAA',
 'ZAD',
 'ZAI',
 'ZC',
 'ZCJ',
 'ZV'}